# Secret Management
- Sometimes accessing data requires that you authenticate to external data sources through JDBC. Instead of directly entering your credentials into a notebook, use Databricks secrets to store your credentials and reference them in notebooks and jobs. To manage secrets, you can use the Databricks CLI to access the Secrets API 2.0.

- We have Two different ways to create Secret:
   - Azure key-vault Backed Secret scope.
   - Databricks-backed secret scope.

## Databricks-backed secret scope

To set up secrets you:
- Create a secret scope(Secret scope names are case insensitive).
- Add secrets to the scope(Secret names are case insensitive).

### Let's Understand the use of secret with an example.

**Step 1: Create Scope with name Freshbooks**

```cmd
databricks secrets create-scope --scope Freshbooks
```

If your account does not have the Premium plan or above, you must create the scope with MANAGE permission granted to all users (“users”).

```cmd
databricks secrets create-scope --scope Freshbooks --initial-manage-principal users
```

**Step 2: Create Secret within scope Freshbooks**

Add the secrets <code>client_id, client_secret</code>, and <code>redirect_url</code>. Run the following commands and enter the secret values in the opened editor.

```cmd
databricks secrets put --scope Freshbooks --key client_id
databricks secrets put --scope Freshbooks --key client_secret
databricks secrets put --scope Freshbooks --key redirect_url
```

## How to Use Secrets In Notebook

- In a notebook, read the secrets that are stored in the secret scope Freshbooks to configure a Freshbooks Api.
- We can   access with secrets with the help of dbutils.
- You can check out the available functions with <code>dbutils.secrets.help()</code>
- You can check out the functionality for each function with with help of <code>dbutils.secrets.help('get')</code>.

In [0]:
dbutils.secrets.help()


Provides utilities for leveraging secrets within notebooks.
Databricks documentation for more info.
 get(scope: String, key: String): String -> Gets the string representation of a secret value with scope and key getBytes(scope: String, key: String): byte[] -> Gets the bytes representation of a secret value with scope and key list(scope: String): Seq -> Lists secret metadata for secrets within a scope listScopes: Seq -> Lists secret scopes

In [0]:
dbutils.secrets.help("get")

/** * Gets the string representation of a secret value with scope and key. * This API assumes the secret is encoded as UTF-8 bytes. This will * always be the case if you use the `string_value` write API. * * Example: dbutils.secrets.get("scope1", "key1") * * @param scope Scope in which the secret was created * @param key Key with which the secret was created */ get(scope: java.lang.String, key: java.lang.String): java.lang.String

**Freshbooks Authorization**

Here You can see how to access with the secrets. 

```python
import requests

base_url = "https://auth.freshbooks.com/oauth/authorize/"
response_type = "code"
redirect_url = dbutils.secrets.get(scope = "Freshbooks", key = "redirect_url")
client_id = dbutils.secrets.get(scope = "Freshbooks", key = "client_id")
client_secrets = dbutils.secrets.get(scope = "Freshbooks", key = "client_secrets")

Url = f"{base_url}?{response_type}{redirect_url}{client_id}{client_secrets}"

response = requests.get(Url)
```

### Grant Access Of Secrets To Another Groups.
- After verifying that the credentials were configured correctly, share these credentials with the freshbooksteam group to use for their analysis.

- Grant the freshbooksteam group read-only permission to these credentials by making the following request.
```cmd
databricks secrets put-acl --scope Freshbooks --principal freshbooksteam --permission READ
```